<a href="https://colab.research.google.com/github/rfflpllcn/ml/blob/master/imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Install Kaggle library
!pip install -q kaggle

# Colab library to upload files to notebook
from google.colab import files
import pandas as pd

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Convolution1D

In [3]:
# Upload kaggle API key file
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!mkdir ~/.kaggle
!touch ~/.kaggle/kaggle.json
!echo '{"username":"michelegiambono","key":"ca6243facbe66d92bf3688aec1a2fd8c"}' > ~/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

# Downlaod data 
!kaggle datasets download -d utathya/imdb-review-dataset -p /content
!unzip \*.zip

 81% 41.0M/50.5M [00:00<00:00, 64.1MB/s]
100% 50.5M/50.5M [00:00<00:00, 93.0MB/s]
Archive:  imdb-review-dataset.zip
  inflating: imdb_master.csv         


In [0]:
def clean_data(df):
  df = df.drop(['Unnamed: 0','type','file'],axis=1)
  df.columns = ["review","sentiment"]
  df = df[df.sentiment!='unsup']
  df['sentiment'] = df['sentiment'].map({'pos': 1, 'neg': 0})
  return df

def get_data():
  df = pd.read_csv('imdb_master.csv', encoding = "ISO-8859-1")
  df = clean_data(df)

  return df.review.values, df.sentiment.values

In [0]:
train_data, train_labels = get_data()

In [0]:
num_words=10000
maxlen = 20
t = Tokenizer(num_words=num_words)
t.fit_on_texts(train_data)

train_data = pad_sequences(t.texts_to_sequences(train_data), maxlen=maxlen)

merely flattening the
embedded sequences and training a single Dense layer on top leads to a model that
treats each word in the input sequence separately, without considering inter-word
relationships and sentence structure (for example, this model would likely treat both
“this movie is a bomb” and “this movie is the bomb” as being negative reviews).


In [0]:
embedding = Embedding(num_words, 8, input_length=maxlen)  # embedding(tf.constant(train_data[0]))

model = Sequential()
model.add(embedding)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model with glove embeddings

see https://www.analyticsvidhya.com/blog/2020/03/pretrained-word-embeddings-nlp/

In [114]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip
!ls
!pwd

--2020-06-02 19:39:43--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-06-02 19:39:43--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-06-02 19:39:44--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [113]:
model.compile(optimizer=optimizers.RMSprop(), loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 20, 8)             80000     
_________________________________________________________________
flatten_12 (Flatten)         (None, 160)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


In [106]:
history = model.fit(train_data, train_labels, batch_size=32, epochs=10, validation_split=0.2)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 3s 81us/step - loss: 0.4463 - acc: 0.7875 - val_loss: 0.4863 - val_acc: 0.7726
Epoch 2/10
40000/40000 [==============================] - 4s 89us/step - loss: 0.2767 - acc: 0.8837 - val_loss: 0.5100 - val_acc: 0.7750
Epoch 3/10
40000/40000 [==============================] - 3s 74us/step - loss: 0.2485 - acc: 0.8964 - val_loss: 0.5053 - val_acc: 0.7874
Epoch 4/10
40000/40000 [==============================] - 3s 65us/step - loss: 0.2336 - acc: 0.9041 - val_loss: 0.5161 - val_acc: 0.7898
Epoch 5/10
40000/40000 [==============================] - 3s 67us/step - loss: 0.2223 - acc: 0.9097 - val_loss: 0.5943 - val_acc: 0.7616
Epoch 6/10
40000/40000 [==============================] - 3s 65us/step - loss: 0.2115 - acc: 0.9150 - val_loss: 0.5150 - val_acc: 0.7980
Epoch 7/10
40000/40000 [==============================] - 3s 65us/step - loss: 0.1997 - acc: 0.9209 - val_loss: 0.5662 - val_acc